In [1]:
import csv
import random

import numpy as np


In [2]:
weights = {
    'blouse': np.array([1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0]),
    'dress': np.array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0,
        0, 0]),
    'outwear': np.array([1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0]),
    'skirt': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0,
        0, 0]),
    'trousers': np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1,
        1, 1])
}

In [3]:
def generate_train_set(source, target, shuffle=False):
    with open(source, newline='') as infile:
        spamreader = csv.reader(infile)
        with open(target, 'w', newline='') as outfile:
            spamwriter = csv.writer(outfile, delimiter=' ')
            
            # keep track of some variables
            head = True
            catagories = {'blouse':[], 'dress':[], 'outwear':[], 'skirt':[], 'trousers':[]}
            image_count = {'blouse':[0, 0], 'dress':[0, 0], 'outwear':[0, 0], 'skirt':[0, 0], 'trousers':[0, 0]}
            
            # process every row
            for row in spamreader:
                # skip the header row
                if head:
                    head = False
                    continue
                    
                bad_data = False

                # parse data
                this_row = [row[0].split('/')[-1] + 'A', row[1], -1, -1, -1, -1]
                for i in range(2, 26):
                    x, y, v = row[i].split('_')
                    
                    # pickout data with invalid points
                    if v == '1' and weights[row[1]][i-2] == 0:
                        bad_data = True
                        break
#                     elif v == '-1' and weights[row[1]][i-2] == 1:
#                         bad_data = True
#                         break
                        
                    if v == '1':
                        this_row += [x, y, 1]
                    else:
                        this_row += [x, y, 0]

                # pickout data without valid normalization points
                if row[1] in ['blouse', 'outwear', 'dress']:
                    if row[7].split('_')[2] == "-1" or row[8].split('_')[2] == "-1":
                        bad_data = True
                else:
                    if row[17].split('_')[2] == "-1" or row[18].split('_')[2] == "-1":
                        bad_data = True
                        
                if bad_data:
                    image_count[row[1]][1] += 1    
                else:
                    catagories[row[1]].append(this_row)
                  
            # calculate total count
            for i in catagories:
                image_count[i][0] = len(catagories[i])

            # write 90% of every catagory as train set
            for i in catagories:
                # shuffle entries as needed
                if (shuffle):
                    random.shuffle(catagories[i])
                train_size = int(len(catagories[i]) * 0.1)
                for row in catagories[i][:train_size]:
                    spamwriter.writerow(row)

            # write 10% of every catagory as valid set
            for i in catagories:
                train_size = int(len(catagories[i]) * 0.1)
                for row in catagories[i][train_size:]:
                    spamwriter.writerow(row)
                    
            return image_count

In [4]:
image_count = generate_train_set("data/train/Annotations/train.csv", 
                                 'data/dataset_valid.txt', shuffle = True)
print(image_count)

{'dress': [4888, 24], 'blouse': [7124, 34], 'skirt': [7602, 16], 'trousers': [6316, 31], 'outwear': [5565, 31]}


In [24]:
total = 0
for k in image_count:
    total += image_count[k][0]

print(total)

31495


In [25]:
count = 0
with open('dataset2.txt', newline='') as infile:
    spamreader = csv.reader(infile)
    for row in spamreader:
        count += 1
        
    print(count)

31495


In [26]:
weights = {}
num_valid = {
    'blouse': 13,
    'outwear': 14,
    'trousers': 7,
    'skirt': 4,
    'dress': 15
}
with open('/home/tqjxlm/dataset/fasionAI/dataset/train/Annotations/train.csv', newline='') as infile:
    spamreader = csv.reader(infile)
    head = True
    
    for row in spamreader:
        if head:
            head = False
            continue
            
        weight = np.zeros(24)

        for i in range(2, 26):
            x, y, v = row[i].split('_')
            if v != '-1':
                weight[i-2] = 1
        
        if row[1] in ['blouse', 'outwear', 'dress']:
            if row[7].split('_')[2] == "-1" or row[8].split('_')[2] == "-1":
                continue
        else:
            if row[17].split('_')[2] == "-1" or row[18].split('_')[2] == "-1":
                continue
        
        if np.count_nonzero(weight) is num_valid[row[1]]:
            weights[row[1]] = weight
        
        if len(weights) is 5:
            break

28345.5